## Проект: надо выделить реплики, где менеджер поздоровался, попрощался, представил себя, упомянул компанию. Затем, надо вывести имена менеджеров и проверить, какие менеджеры поздоровались и попрощались с клиентом

In [ ]:
import pandas as pd
import numpy as np
import re
from pymystem3 import Mystem
m = Mystem()
from nltk.corpus import wordnet
from gensim.models import Word2Vec
import gensim
import pymorphy2

#### Для начала считаем файлы в ноутбук с помощью метода read_csv

In [4]:
df = pd.read_csv("test_data.csv")
df

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


#### EDA

In [5]:
df.role.unique()

array(['client', 'manager'], dtype=object)

In [6]:
df[df.line_n == 0]

,dlg_id,line_n,role,text
0,0,0,client,Алло
109,1,0,client,Да здравствуйте когда заканчивается
164,2,0,client,Алло
249,3,0,client,Добрый день
302,4,0,client,Алло
337,5,0,client,Алло


In [7]:
df[df.role == 'manager'].text

1                                      Алло здравствуйте
3      Меня зовут ангелина компания диджитал бизнес з...
5      Угу ну возможно вы рассмотрите и другие вариан...
8          Угу а на что вы обращаете внимание при выборе
11                                 Что для вас приоритет
                             ...                        
472                                          Так дмитрий
473               Все записала тогда завтра ждите звонка
475    По поводу виджетов и с ними уже обсудите конкр...
476    Все я вам высылаю счет и с вами на связи если ...
479                       Ну до свидания хорошего вечера
Name: text, Length: 201, dtype: object

#### Чтобы найти синонимы слов, для начала провекторизуем слова с помощью word2vec. К счастью, в библиотеке gensim можно с помощью api.load скачать русско язычный набор слов

In [8]:
import gensim.downloader as api

model = api.load("word2vec-ruscorpora-300")

[==================================================] 100.0% 198.8/198.8MB downloaded


#### Проведем для начала предобработку данных, то есть уменьшим все заглавные буквы на прописные и приведем к изначальной морфологической форме с помощью метода MorphAnalyzer - библиотеки pymorphy2. А результат предобработки запишием в data

In [9]:
lemmatizer = pymorphy2.MorphAnalyzer(lang='ru')
def preprocessing(sentence):
    sentence = sentence.lower()
    tokenized_sentence = sentence.split()
    return ' '.join(lemmatizer.parse(word)[-1].normal_form for word in tokenized_sentence)
data = df[df.role == 'manager'].text.apply(preprocessing)

In [10]:
# Все это добро сохраним в data
data = pd.DataFrame(data)

#### В последующей ячейки я находил синонимы к слову здравствовать(здравствуйте), звать(меня зовут) и компания(фирма, фабрика и т.д.) и занимался поиском этих же слов с синонимами в датафрейме. Синонимами являются именно те слова, которые являются близкими к ним по расстоянию. В данном случае, я использовал косинусное расстояние между векторами. Если находил либо синонимы, либо сами слова, отмечал 1, а в тех, где не находил, отмечал 0

#### Для "прощания" я не стал искать, так как менеджеры всегда говорят "до свидания" или "до встречи", так как встреча и свидание никак не связаны с прощанием, к тому же, до встречи это уже два слова.

In [11]:
# Функция находящее пересечение между двумя списками. Она нужна для того, 
# чтобы найти есть ли хотя бы одно слово из списка синонимов в предложении
def intersection(lst1, lst2):
  lst3 = [value for value in lst1 if value in lst2]
  return lst3
# Здесь я нахожу синонимы к слову здравствовать
words = ['здравствовать_VERB']
lst_inf = []
for word in words:
  if word in model:
    print(word)
    lst_inf.append(word.split('_')[0])
    for i in model.most_similar(positive=[word], topn=2):
      print(i[0], i[1])
      lst_inf.append(i[0].split('_')[0])

# Функция для проверки датасета(содержит ли датасет приветсвие или хотя бы синонимы этого слова)
def is_contains_greetings(sentence):
  lst = intersection(sentence.split(' '), lst_inf)
  if len(lst) > 0 or 'добрый деть' in sentence:
    return 1
  return 0
data['is_greets'] = data.text.apply(is_contains_greetings)

# Функция для проверки датасета(содержит ли датасет прощание или хотя бы синонимы этого слова)
def is_contains_parting(sentence):
  if 'встреча' in sentence or 'свидание' in sentence:
    return 1
  return 0
data['is_parting'] = data.text.apply(is_contains_parting)

# Здесь я нахожу синонимы к слову звать
words = ['звать_VERB']
lst_inf = []
for word in words:
  if word in model:
    print(word)
    lst_inf.append(word.split('_')[0])
    for i in model.most_similar(positive=[word], topn=2):
      print(i[0], i[1])
      lst_inf.append(i[0].split('_')[0])

# Функция для проверки датасета(содержит ли датасет представление менеджера или хотя бы синонимы этого слова)
def is_contains_only_syns(sentence):
  lst = intersection(sentence.split(' '), lst_inf)
  if len(lst) > 0:
    return 1
  return 0
data['is_shows_himself'] = data.text.apply(is_contains_only_syns)

# Здесь я нахожу синонимы к слову компания
words = ['компания_NOUN']
lst_inf = []
for word in words:
  if word in model:
    print(word)
    lst_inf.append(word.split('_')[0])
    for i in model.most_similar(positive=[word], topn=8):
      print(i[0], i[1])
      lst_inf.append(i[0].split('_')[0])

data['is_noticed_company'] = data.text.apply(is_contains_only_syns)

здравствовать_VERB
приветствовать_VERB 0.5425398945808411
привет_NOUN 0.5378391146659851
звать_VERB
кликать_VERB 0.669928789138794
позвать_VERB 0.641196608543396
компания_NOUN
фирма_NOUN 0.6828166246414185
holding_NOUN 0.6733368039131165
корпорация_NOUN 0.6696858406066895
exxon::mobil_NOUN 0.6678482294082642
холдинг_NOUN 0.6663289070129395
uc::rusal_NOUN 0.665887713432312
holdings_NOUN 0.6658523678779602
schlumberger_NOUN 0.6637731194496155


#### Таким образом, мы получим текущий датасет состоящий из реплик менеджеров, содержащих приветсвие, представление самого себя, прощания и упоминания компании

In [12]:
data[data.is_greets == 1]

,text,is_greets,is_parting,is_shows_himself,is_noticed_company
1,алло здравствовать,1,0,0,0
110,алло здравствовать,1,0,0,0
166,алло здравствовать,1,0,0,0
250,алло дмитрия добрый деть,1,0,0,0


In [13]:
data[data.is_noticed_company == 1]

,text,is_greets,is_parting,is_shows_himself,is_noticed_company
3,я звать ангелина компания диджитал бизнес звон...,0,0,1,1
111,я звать ангелина компания диджитал бизнес звон...,0,0,1,1
167,я звать ангелина компания диджитал бизнес звон...,0,0,1,1
251,добрый я максим звать компания китобизнес удоб...,0,0,1,1
268,так а вот подсказать пожалуйста может быть про...,0,0,0,1
273,транспортный компания хороший а у вы узкий вот...,0,0,0,1


#### Здесь я нахожу имена менеджеров в репликах, где они представились

In [14]:
# Я действовал по такой логике, если в предложении содержится слово звать или я, то скорее всего менеджер назвал свое имя
names = []
for i in np.array(data[data.is_shows_himself == 1].text):
  lst = i.split(' ')
  for j in range(len(lst)):
    if lst[j] == 'я':
      if lst[j + 1] == 'звать':
        names.append(lst[j + 2])

      else:
        names.append(lst[j + 1])

lst_names = []
i = 0
for num in np.array(data.is_shows_himself):
  if num == 1:
    lst_names.append(names[i])
    i+=1
  else:
    lst_names.append("No_name")

# Также все это я сохраняю в data
data['name_of_manager'] = lst_names

#### Данные из data записываем в изначальный датафрейм

In [15]:
array = np.array(df.role)
greets = np.array(data['is_greets'])
parting = np.array(data['is_parting'])
shows_himself = np.array(data['is_shows_himself'])
notices_company = np.array(data['is_noticed_company'])
manager_name = np.array(data['name_of_manager'])
lst_greets, lst_parting, lst_shows_himself, lst_company, lst_name_manager = [], [], [], [], []
j = 0
for i in array:
  if i == 'manager':
    lst_greets.append(greets[j])
    lst_parting.append(parting[j])
    lst_shows_himself.append(shows_himself[j])
    lst_company.append(notices_company[j])
    lst_name_manager.append(manager_name[j])
    j+=1
  else:
    lst_greets.append(0)
    lst_parting.append(0)
    lst_shows_himself.append(0)
    lst_company.append(0)
    lst_name_manager.append("No_name")

df['greeting'] = lst_greets
df['parting'] = lst_parting
df['is_introduces_himself'] = lst_shows_himself
df['is_noticed_company_name'] = lst_company
df['name_of_manager'] = lst_name_manager

#### Реплики где менеджер поздаровался

In [16]:
df[(df.greeting == 1) & (df.role == 'manager')].text

1             Алло здравствуйте
110           Алло здравствуйте
166           Алло здравствуйте
250    Алло дмитрий добрый день
Name: text, dtype: object

#### Реплики где менеджер представил себя

In [17]:
df[(df.is_introduces_himself == 1) & (df.role == 'manager')].text

3      Меня зовут ангелина компания диджитал бизнес з...
111    Меня зовут ангелина компания диджитал бизнес з...
167    Меня зовут ангелина компания диджитал бизнес з...
251    Добрый меня максим зовут компания китобизнес у...
Name: text, dtype: object

#### Имена менеджеров

In [18]:
df[(df.is_introduces_himself == 1) & (df.role == 'manager')][['text', 'name_of_manager']]

,text,name_of_manager
3,Меня зовут ангелина компания диджитал бизнес з...,ангелина
111,Меня зовут ангелина компания диджитал бизнес з...,ангелина
167,Меня зовут ангелина компания диджитал бизнес з...,ангелина
251,Добрый меня максим зовут компания китобизнес у...,максим


#### Реплики где менеджер назвал компанию

In [19]:
df[(df.is_noticed_company_name == 1) & (df.role == 'manager')].text

3      Меня зовут ангелина компания диджитал бизнес з...
111    Меня зовут ангелина компания диджитал бизнес з...
167    Меня зовут ангелина компания диджитал бизнес з...
251    Добрый меня максим зовут компания китобизнес у...
268    Так а вот подскажите пожалуйста может быть про...
273    Транспортная компания хорошо а у вас уже вот в...
Name: text, dtype: object

#### Реплики где менеджер попрощался

In [20]:
df[(df.parting == 1) & (df.role == 'manager')].text

108                           Всего хорошего до свидания
163                                          До свидания
335    Во вторник все ну с вами да тогда до вторника ...
479                       Ну до свидания хорошего вечера
Name: text, dtype: object

In [21]:
df[df.line_n == 0]

,dlg_id,line_n,role,text,greeting,parting,is_introduces_himself,is_noticed_company_name,name_of_manager
0,0,0,client,Алло,0,0,0,0,No_name
109,1,0,client,Да здравствуйте когда заканчивается,0,0,0,0,No_name
164,2,0,client,Алло,0,0,0,0,No_name
249,3,0,client,Добрый день,0,0,0,0,No_name
302,4,0,client,Алло,0,0,0,0,No_name
337,5,0,client,Алло,0,0,0,0,No_name


#### Менеджеры, которые удовлетворяют требованию(поздоровались и попрощались с клиентом)

In [22]:
lst_start = []
lst_end = []
lst_requirement = []
arr = np.array(df[df.line_n == 0].index)
for i in range(len(arr)):
  if i % 2 == 0:
    lst_start.append(arr[i])
  else:
    lst_end.append(arr[i])

for i in range(len(lst_start)):
  if len(df.iloc[lst_start[0]:lst_end[0]][df.iloc[lst_start[0]:lst_end[0]].greeting == 1]) > 0 \
      and len(df.iloc[lst_start[0]:lst_end[0]][df.iloc[lst_start[0]:lst_end[0]].parting == 1]):
      lst_requirement.append(df.iloc[lst_start[0]:lst_end[0]][df.iloc[lst_start[0]:lst_end[0]].name_of_manager != "No_name"].name_of_manager.iloc[0])

for i in lst_requirement:
  print(f'{i} выплонила требование')

ангелина выплонила требование
ангелина выплонила требование
ангелина выплонила требование


## Пробовал вывести имена с помощью named entity. Модель не смогла вывести все имена